In [41]:
import requests
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None
import numpy as np
import datetime
from datetime import date
import time
from urllib.request import Request, urlopen

import urllib.request
from bs4 import BeautifulSoup
import re

import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.client import GoogleCredentials
from google.oauth2.service_account import Credentials

scopes = ['https://www.googleapis.com/auth/spreadsheets', 'https://www.googleapis.com/auth/drive']

credentials = {'CREDENTIALS_HERE'}
gc = gspread.service_account_from_dict(credentials)

In [42]:
wb = gc.open_by_url('https://docs.google.com/spreadsheets/d/1mUbB48_bVclseWFWOnMyfsrIKhPGUP3VmpO6JZZPFu8/edit#gid=250866955')
sheet = wb.worksheet('Contacts')

In [43]:
before_df = pd.DataFrame(sheet.get_all_records())

In [44]:
before_df.head(20)

,Company ID,Company name,Contact Information - Name,Role,Contact Information - Phone Number,Contact Information - Email
0,8136886917,0 BOONE COUNTY FIRE PROTECTION DIST,Scott Olsen,Fire Chief,(573) 447 - 5000 Ext. 5001,solsen@bcfdmo.com
1,8136886917,0 BOONE COUNTY FIRE PROTECTION DIST,Chuck Leake,Director/Program Manager,(573) 447 - 5000 Ext. 5050,cleake@bcfdmo.com
2,8136909750,Health & Safety Institute - Grand Rapids,Nicole Printup,Accreditation Manager,(800) 800 - 7099,nprintup@hsi.com
3,8136909749,3 AMERICAN MEDICAL RESPONSE,Lisa Higuchi,Clinical / Education Manager,(951) 782 - 5294,Lisa.Higuchi@amr.net
4,8136259997,Abrams College,Daniel Lucky,Nurse Practitioner,(209) 527 - 7777,dlucky@abramscollege.com
5,8136922415,ABSOLUTE PERSONAL PROTECTION,Brandon Ramsey,Instructor,NA,absolutepersonalprotection@gmail.com
6,8136260270,Academy for Professional Development,Vince Inaudi,Chief Academic Officer,(510) 888 - 0349,vince@afpdus.com
7,8136892505,Accredited EMS Fire Training Inc,Steve Bristow,President,(925) 708 - 5377,info@accreditedemsfiretraining.com
8,8136301374,"ADAIR COUNTY AMBULANCE DISTRICT ""THE ACADEMY""",Larry Burton,Executive Director,NA,burton93356@yahoo.com
9,8136927771,Alhambra Unified School District,Leann Huang,Technical Education Specialist,(626) 943 - 6990,huang_leann@ausd.us


In [45]:
after_df = pd.DataFrame()

for i in range(len(before_df)):
    comp_id = before_df['Company ID'][i]
    comp_name = before_df['Company name'][i]
    names_list = before_df['Contact Information - Name'][i].split('\n')
    role_list = before_df['Role'][i].split('\n')
    phone_list = before_df['Contact Information - Phone Number'][i].split('\n')
    email_list = before_df['Contact Information - Email'][i].split('\n')

    for j in range(len(names_list)):
        after_df = after_df.append({'Company ID':comp_id, 'Company Name':comp_name, 'Contact Name':names_list[j], 'Role':role_list[j], 'Phone Number':phone_list[j], 'Email':email_list[j]}, ignore_index=True)

after_df = after_df.replace('NA', '')

In [52]:
after_sheet = wb.worksheet('Contacts to import')
after_sheet.clear()
set_with_dataframe(after_sheet, after_df)